In [ ]:
import subprocess
import os, time, json
from tqdm import tqdm
import numpy as np
from nltk.translate.bleu_score import sentence_bleu


def computeBleu1_to_4(reference_list, candidate_list, filename, i):
    
    bleu1_sum = bleu2_sum = bleu3_sum = bleu4_sum = bleuA_sum = 0
        
    for (ref, cand) in zip(reference_list, candidate_list):
        
        tokens_real = ref.split(' ')
        tokens_pred = cand.split(' ')

        if cand == '':
            bleu1_score = bleu2_score = bleu3_score = bleu4_score = bleuA_score = 0
            
        else:
            bleu1_score = sentence_bleu([tokens_real], tokens_pred, weights=(1.0, 0.0, 0.0, 0.0))
            bleu2_score = sentence_bleu([tokens_real],tokens_pred, weights=(0.0, 1.0, 0.0, 0.0))
            bleu3_score = sentence_bleu([tokens_real],tokens_pred, weights=(0.0, 0.0, 1.0, 0.0))
            bleu4_score = sentence_bleu([tokens_real],tokens_pred, weights=(0.0, 0.0, 0.0, 1.0))
            bleuA_score = sentence_bleu([tokens_real], tokens_pred, weights=(0.25, 0.25, 0.25, 0.25))

        bleu1_sum += bleu1_score
        bleu2_sum += bleu2_score
        bleu3_sum += bleu3_score
        bleu4_sum += bleu4_score
        bleuA_sum += bleuA_score 

    output = 'BLEU_[A-1-2-3-4]: {}/{}/{}/{}/{}'.format(
        round(bleuA_sum/len(reference_list), 3) * 100,
        round(bleu1_sum/len(reference_list), 3) * 100, 
        round(bleu2_sum/len(reference_list), 3) * 100, 
        round(bleu3_sum/len(reference_list), 3) * 100,
        round(bleu4_sum/len(reference_list), 3) * 100
    )
        
    filename.write('[TOKENS {}]: {}\n'.format(i,output))
    
def levenshtein(seq1, seq2):
    
    size_x = len(seq1) + 1
    size_y = len(seq2) + 1
    matrix = np.zeros ((size_x, size_y))
    for x in range(size_x):
        matrix [x, 0] = x
    for y in range(size_y):
        matrix [0, y] = y

    for x in range(1, size_x):
        for y in range(1, size_y):
            if seq1[x-1] == seq2[y-1]:
                matrix [x,y] = min(
                    matrix[x-1, y] + 1,
                    matrix[x-1, y-1],
                    matrix[x, y-1] + 1
                )
            else:
                matrix [x,y] = min(
                    matrix[x-1,y] + 1,
                    matrix[x-1,y-1] + 1,
                    matrix[x,y-1] + 1
                )
                
    return (matrix[size_x - 1, size_y - 1])



In [ ]:
def computePerfectPrediction(predictions_file, target_file):

    perfect_pred = {}


    target = open(target_file,'r')
    target_list =  target.readlines()

    for i in range(1,16):

        perfect_pred[i] = 0

        with open(predictions_file) as fread:

            data = fread.readlines()

            for (ref,pred) in zip(target_list,data):

                pred = pred.strip().lower().split()
                pred = pred[0:i]

                ref = ref.strip().lower().split()
                ref = ref[0:i]

                if len(ref)>=i and len(pred)>=i:
                    if ''.join(pred) == ''.join(ref):
                        perfect_pred[i]+=1

            len_tokens_subset = len([item.strip() for item in target_list if len(item.split())>=i ])
            perfect_pred[i] = (perfect_pred[i], len_tokens_subset)
            
    return perfect_pred



def feasibleBleuOnly(predictions_file, target_file, task='javadoc'):
    
    if task=='overall':
        
        references_list = target_file
        candidates_list = predictions_file
        
        for i in range(4,16):

            references = []
            candidates = []
            
            for (ref,pred) in zip(references_list,candidates_list):
                
                if len(ref.split())>=i:
                
                    pred = pred.strip().lower()
                    ref = ref.strip().lower()
                    references.append(ref)
                    candidates.append(pred)
            
            filename  = open('BLEU_result_{}.txt'.format(task),'a+')
            computeBleu1_to_4(references, candidates,filename, i)
            filename.close()
            
    else:
        target = open(target_file,'r')
        target_list =  target.readlines()

        for i in range(4,16):

            with open(predictions_file) as fread:

                data = fread.readlines()

                references = []
                candidates = []

                for (ref,pred) in zip(target_list,data):

                    if len(ref.split())>=i:

                        pred = pred.strip().lower()
                        ref = ref.strip().lower()
                        references.append(ref)
                        candidates.append(pred)

                filename  = open('BLEU_result_{}.txt'.format(task),'a+')
                computeBleu1_to_4(references, candidates,filename, i)
                filename.close()

                
def levenshteinOverList(predictions_file, references_file , task='javadoc'):
    
    if task=='overall':
        target_list = references_file
        data = predictions_file
    
    else:
        p = open(predictions_file,'r')
        data = p.readlines()

        r = open(references_file,'r')
        target_list = r.readlines()
        
    references = {}
    candidates = {}
    lev_list = {}
    
    for i in range(1,16):
        references[i]=[]
        candidates[i]=[]
        lev_list[i]=0
    
    for i in range(1,16):

        for (ref,pred) in zip(target_list,data):
            
            target_lev = len(ref.strip().split())
            
            if target_lev >=i:
                
                pred = pred.strip().lower().split()[0:i]
                ref = ref.strip().lower().split()[0:i]
                references[i].append(ref)
                candidates[i].append(candidates)
                lev_list[i]+=levenshtein(ref, pred)
                
    for i in range(1,16):
        
        filename  = open('LEV_result_{}.txt'.format(task),'a+')
        lev = lev_list[i]
        output = 'LEV: {}'.format(round(lev/len(references[i]),3))
        print('[{}] len of references: {}'.format(i,len(references[i])))
        filename.write('[TOKENS {}]: {}\n'.format(i,output))
        filename.close()

In [ ]:
###### PERFECT PREDICTIONS #######


####### Test set ############

# path_to_predictions_jdoc = 
# path_to_targets_jdoc = 

#############################



####### Hyperparameter #######

# path_to_predictions_jdoc = 
# path_to_targets_jdoc = 

#############################


hp_num = 0
hp_den = 0

pred_javadoc = computePerfectPrediction(path_to_predictions_jdoc, path_to_targets_jdoc)
perfect_predictions_javadoc = {}
for i in range(1,16):        
    numbers, len_data = pred_javadoc[i]
    percentage = (numbers / len_data) * 100
    print('[@{} tokens]: Perfect Javadoc: {}% ({}/{})'.format(i, percentage, numbers, len_data))
    perfect_predictions_javadoc[i] = {'percentage':percentage, '#perfect':numbers, '#tokens':len_data}

    #hp_num += numbers
    #hp_den += len_data

# print('javadoc hp results: {}'.format((hp_num/hp_den)*100))

print('\n\n******************************************************************\n\n')



####### Test set ############

# path_to_predictions_inside = 
# path_to_targets_inside = 

#############################


####### Hyperparameter #######

# path_to_predictions_inside = 
# path_to_targets_inside = 

#############################


pred_inside = computePerfectPrediction(path_to_predictions_inside, path_to_targets_inside)

hp_num = 0
hp_den = 0

perfect_predictions_inside = {}

for i in range(1,16):        
    
    numbers, len_data = pred_inside[i]
    percentage = (numbers / len_data) * 100
    print('[@{} tokens]: Perfect Inside: {}% ({}/{})'.format(i, percentage, numbers, len_data))
    perfect_predictions_inside[i] = {'percentage':percentage, '#perfect':numbers, '#tokens':len_data}
    
    #hp_num += numbers
    #hp_den += len_data

# print('inside hp results: {}'.format((hp_num/hp_den)*100))
    
print('\n\n******************************************************************\n\n')



for i in range(1,16):
    
    javadoc_item = perfect_predictions_javadoc[i]
    numbers_javadoc = javadoc_item['#perfect']
    len_data_javadoc = javadoc_item['#tokens']
    
    inside_item = perfect_predictions_inside[i]
    numbers_inside = inside_item['#perfect']
    len_data_inside = inside_item['#tokens']
        
    percentage = ( (numbers_javadoc + numbers_inside) / (len_data_javadoc + len_data_inside) * 100 )
    
    
    print('[@{} tokens]: Perfect Overall: {}% ({}/{})'.format(i, percentage, numbers_inside+numbers_javadoc, 
                                                              len_data_javadoc+len_data_inside))

In [ ]:
###### BLEU #####

references_overall = []
candidates_overall = []

with open(path_to_predictions_inside,'r') as fread:
    for item in fread.readlines():
        candidates_overall.append(item.strip().lower())

with open(path_to_targets_inside,'r') as fread:
    for item in fread.readlines():
        references_overall.append(item.strip().lower())
        
with open(path_to_predictions_jdoc,'r') as fread:
    for item in fread.readlines():
        candidates_overall.append(item.strip().lower())

with open(path_to_targets_jdoc,'r') as fread:
    for item in fread.readlines():
        references_overall.append(item.strip().lower())
        

feasibleBleuOnly(path_to_predictions_inside, path_to_targets_inside, 'inside')
feasibleBleuOnly(path_to_predictions_jdoc, path_to_targets_jdoc, 'javadoc')
feasibleBleuOnly(candidates_overall, references_overall, 'overall')

In [ ]:
###### LEVENSHTEIN ######

levenshteinOverList(path_to_predictions_inside, path_to_targets_inside, 'inside')
levenshteinOverList(path_to_predictions_jdoc, path_to_targets_jdoc, 'javadoc')
levenshteinOverList(candidates_overall, references_overall, 'overall')